In [1]:
import gc
import pathlib

In [2]:
import numpy as np
import pandas as pd

In [3]:
def reduce_memory_usage(df, verbose=True):
    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in ("int16", "int32", "int64"):
            c_min = df[col].min()
            c_max = df[col].max()

            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                df[col] = df[col].astype(np.int64)

        elif col_type in ("float16", "float32", "float64"):
            c_min = df[col].min()
            c_max = df[col].max()

            if (
                c_min > np.finfo(np.float16).min
                and c_max < np.finfo(np.float16).max
            ):
                # df[col] = df[col].astype(np.float16)
                pass
            elif (
                c_min > np.finfo(np.float32).min
                and c_max < np.finfo(np.float32).max
            ):
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)

In [4]:
train_days = 1913
test_days = 28

In [5]:
root_dir_path = pathlib.Path("..")
data_dir_path = root_dir_path / "data"
raw_dir_path = data_dir_path / "raw"
calendar_path = raw_dir_path / "calendar.csv"
sales_train_validation_path = raw_dir_path / "sales_train_validation.csv"
sell_prices_path = raw_dir_path / "sell_prices.csv"
sample_submission_path = raw_dir_path / "sample_submission.csv"
processed_dir_path = data_dir_path / "processed"
train_path = processed_dir_path / "train.parquet"
test_path = processed_dir_path / "test.parquet"

In [6]:
calendar = pd.read_csv(calendar_path, parse_dates=["date"])
sales_train_validation = pd.read_csv(sales_train_validation_path)
sell_prices = pd.read_csv(sell_prices_path)
sample_submission = pd.read_csv(sample_submission_path)

### calendar

In [7]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [8]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          1969 non-null   datetime64[ns]
 1   wm_yr_wk      1969 non-null   int64         
 2   weekday       1969 non-null   object        
 3   wday          1969 non-null   int64         
 4   month         1969 non-null   int64         
 5   year          1969 non-null   int64         
 6   d             1969 non-null   object        
 7   event_name_1  162 non-null    object        
 8   event_type_1  162 non-null    object        
 9   event_name_2  5 non-null      object        
 10  event_type_2  5 non-null      object        
 11  snap_CA       1969 non-null   int64         
 12  snap_TX       1969 non-null   int64         
 13  snap_WI       1969 non-null   int64         
dtypes: datetime64[ns](1), int64(7), object(6)
memory usage: 215.5+ KB


In [9]:
calendar.describe(include="all")

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
count,1969,1969.000000,1969,1969.000000,1969.000000,1969.000000,1969,162,162,5,5,1969.000000,1969.000000,1969.000000
unique,1969,NaN,7,NaN,NaN,NaN,1969,30,4,4,2,NaN,NaN,NaN
top,2014-07-27 00:00:00,NaN,Saturday,NaN,NaN,NaN,d_1377,SuperBowl,Religious,Father's day,Cultural,NaN,NaN,NaN
freq,1,NaN,282,NaN,NaN,NaN,1,6,55,2,4,NaN,NaN,NaN
first,2011-01-29 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2016-06-19 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,11347.086338,NaN,3.997461,6.325546,2013.288471,NaN,NaN,NaN,NaN,NaN,0.330117,0.330117,0.330117
std,NaN,155.277043,NaN,2.001141,3.416864,1.580198,NaN,NaN,NaN,NaN,NaN,0.470374,0.470374,0.470374
min,NaN,11101.000000,NaN,1.000000,1.000000,2011.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
25%,NaN,11219.000000,NaN,2.000000,3.000000,2012.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000


### sales_train_validation

In [10]:
sales_train_validation.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [11]:
sales_train_validation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1919 entries, id to d_1913
dtypes: int64(1913), object(6)
memory usage: 446.4+ MB


In [12]:
sales_train_validation.describe(include="all")

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
count,30490,30490,30490,30490,30490,30490,30490.000000,30490.000000,30490.000000,30490.000000,...,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000
unique,30490,3049,7,3,10,3,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,HOUSEHOLD_2_014_WI_3_validation,FOODS_1_050,FOODS_3,FOODS,CA_2,CA,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,10,8230,14370,3049,12196,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,1.070220,1.041292,0.780026,0.833454,...,1.370581,1.586159,1.693670,1.248245,1.232207,1.159167,1.149000,1.328862,1.605838,1.633158
std,NaN,NaN,NaN,NaN,NaN,NaN,5.126689,5.365468,3.667454,4.415141,...,3.740017,4.097191,4.359809,3.276925,3.125471,2.876026,2.950364,3.358012,4.089422,3.812248
min,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000


In [13]:
sales_train_validation.duplicated(["item_id", "store_id"]).sum()

0

In [14]:
sales_train_validation["id"].str.endswith("_validation").sum()

30490

In [15]:
sales_train_validation["id"].str.endswith("_evaluation").sum()

0

### sell_prices

In [16]:
sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [17]:
sell_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   store_id    object 
 1   item_id     object 
 2   wm_yr_wk    int64  
 3   sell_price  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 208.8+ MB


In [18]:
sell_prices.describe(include="all")

,store_id,item_id,wm_yr_wk,sell_price
count,6841121,6841121,6.841121e+06,6.841121e+06
unique,10,3049,NaN,NaN
top,TX_2,HOUSEHOLD_1_234,NaN,NaN
freq,701214,2820,NaN,NaN
mean,NaN,NaN,1.138294e+04,4.410952e+00
std,NaN,NaN,1.486100e+02,3.408814e+00
min,NaN,NaN,1.110100e+04,1.000000e-02
25%,NaN,NaN,1.124700e+04,2.180000e+00
50%,NaN,NaN,1.141100e+04,3.470000e+00
75%,NaN,NaN,1.151700e+04,5.840000e+00


### sample_submission

In [19]:
sample_submission.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60980 entries, 0 to 60979
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      60980 non-null  object
 1   F1      60980 non-null  int64 
 2   F2      60980 non-null  int64 
 3   F3      60980 non-null  int64 
 4   F4      60980 non-null  int64 
 5   F5      60980 non-null  int64 
 6   F6      60980 non-null  int64 
 7   F7      60980 non-null  int64 
 8   F8      60980 non-null  int64 
 9   F9      60980 non-null  int64 
 10  F10     60980 non-null  int64 
 11  F11     60980 non-null  int64 
 12  F12     60980 non-null  int64 
 13  F13     60980 non-null  int64 
 14  F14     60980 non-null  int64 
 15  F15     60980 non-null  int64 
 16  F16     60980 non-null  int64 
 17  F17     60980 non-null  int64 
 18  F18     60980 non-null  int64 
 19  F19     60980 non-null  int64 
 20  F20     60980 non-null  int64 
 21  F21     60980 non-null  int64 
 22  F22     60980 non-null

In [21]:
sample_submission.describe(include="all")

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
count,60980,60980.0,60980.0,60980.0,60980.0,60980.0,60980.0,60980.0,60980.0,60980.0,...,60980.0,60980.0,60980.0,60980.0,60980.0,60980.0,60980.0,60980.0,60980.0,60980.0
unique,60980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,HOUSEHOLD_2_014_WI_3_validation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Preprocessing

In [22]:
reduce_memory_usage(calendar)
reduce_memory_usage(sales_train_validation)
reduce_memory_usage(sell_prices)
reduce_memory_usage(sample_submission)

In [23]:
calendar.drop(columns=["weekday", "wday", "month", "year"], inplace=True)

In [24]:
# sales_train_validation.drop(columns=["dept_id", "cat_id", "state_id"], inplace=True)

In [25]:
intermediate = sample_submission["id"].str.extract(r"(?P<item_id>\w+_\d+_\d+)_(?P<store_id>\w+_\d+)_\w+")
sample_submission = pd.concat([sample_submission, intermediate], axis=1)
intermediate = sample_submission["item_id"].str.extract(r"(?P<dept_id>\w+_\d+)_\d+")
sample_submission = pd.concat([sample_submission, intermediate], axis=1)
intermediate = sample_submission["dept_id"].str.extract(r"(?P<cat_id>\w+)_\d+")
sample_submission = pd.concat([sample_submission, intermediate], axis=1)
intermediate = sample_submission["store_id"].str.extract(r"(?P<state_id>\w+)_\d+")
sample_submission = pd.concat([sample_submission, intermediate], axis=1)

In [26]:
del intermediate

In [27]:
gc.collect()

24

In [28]:
sample_submission.shape

(60980, 34)

In [29]:
# sample_submission.drop(columns=["dept_id", "cat_id", "state_id"], inplace=True)

### train

In [30]:
train = sales_train_validation.melt(id_vars=["id", "item_id", "store_id", "dept_id", "cat_id", "state_id"], var_name="d", value_name="demand")

In [31]:
del sales_train_validation

In [32]:
gc.collect()

0

In [33]:
train.shape

(58327370, 8)

In [34]:
train = train.merge(calendar, copy=False, how="left", on="d")
train = train.merge(sell_prices, copy=False, how="left", on=["store_id", "item_id", "wm_yr_wk"])

In [35]:
train.shape

(58327370, 18)

In [36]:
train.head()

,id,item_id,store_id,dept_id,cat_id,state_id,d,demand,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,CA_1,HOBBIES_1,HOBBIES,CA,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,CA_1,HOBBIES_1,HOBBIES,CA,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,CA_1,HOBBIES_1,HOBBIES,CA,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,CA_1,HOBBIES_1,HOBBIES,CA,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,CA_1,HOBBIES_1,HOBBIES,CA,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN,0,0,0,NaN


In [37]:
train.to_parquet(train_path)

### test

In [38]:
test = sample_submission.melt(id_vars=["id", "item_id", "store_id", "dept_id", "cat_id", "state_id"], var_name="d", value_name="demand")

In [39]:
del sample_submission

In [40]:
gc.collect()

0

In [41]:
test.shape

(1707440, 8)

In [42]:
is_evaluation = test["id"].str.endswith("_evaluation")
intermediate = test["d"].str[1:]
intermediate = intermediate.astype("int32")
intermediate += train_days
intermediate[is_evaluation] += test_days
test["d"] = intermediate.apply("d_{}".format)

In [43]:
del is_evaluation
del intermediate

In [44]:
gc.collect()

36

In [45]:
test.drop(columns="demand", inplace=True)

In [46]:
test = test.merge(calendar, copy=False, how="left", on="d")
test = test.merge(sell_prices, copy=False, how="left", on=["store_id", "item_id", "wm_yr_wk"])

In [47]:
test.shape

(1707440, 17)

In [48]:
test.head()

,id,item_id,store_id,dept_id,cat_id,state_id,d,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,CA_1,HOBBIES_1,HOBBIES,CA,d_1914,2016-04-25,11613,NaN,NaN,NaN,NaN,0,0,0,8.38
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,CA_1,HOBBIES_1,HOBBIES,CA,d_1914,2016-04-25,11613,NaN,NaN,NaN,NaN,0,0,0,3.97
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,CA_1,HOBBIES_1,HOBBIES,CA,d_1914,2016-04-25,11613,NaN,NaN,NaN,NaN,0,0,0,2.97
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,CA_1,HOBBIES_1,HOBBIES,CA,d_1914,2016-04-25,11613,NaN,NaN,NaN,NaN,0,0,0,4.64
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,CA_1,HOBBIES_1,HOBBIES,CA,d_1914,2016-04-25,11613,NaN,NaN,NaN,NaN,0,0,0,2.88


In [49]:
test.to_parquet(test_path)